# Intro

This notebook aims at downloading sequences infos(images and bboxes) from a Distant/Real Pyronear Alert API isntance in order to recreate alerts/sequences with ease in local dev environnement

You need to have accesses to a distant api and provide ```DISTANT_API_URL```, ```DISTANT_ALERT_API_LOGIN```, ```DISTANT_ALERT_API_PASSWORD```

And obvisouly, fill ```SEQUENCE_ID_LIST``` with the sequence id you want in order to download related data


If you do not have anything from above, do not worry, you can use the notebook "send_real_alerts" to send some alerts to local env dev

In [ ]:
SEQUENCE_ID_LIST = [13802, 9456]
BASE_DIRECTORY = "alerts" # directory where to put sequences data

In [ ]:
import requests
from dotenv import load_dotenv
import os

from pyroclient import Client

load_dotenv("../.env")
DISTANT_API_URL = os.environ.get("DISTANT_API_URL")
DISTANT_ALERT_API_LOGIN = os.environ.get("DISTANT_ALERT_API_LOGIN")
DISTANT_ALERT_API_PASSWORD = os.environ.get("DISTANT_ALERT_API_PASSWORD")

In [ ]:
token = requests.post(
    f"{DISTANT_API_URL}api/v1/login/creds",
    data={"username": DISTANT_ALERT_API_LOGIN, "password": DISTANT_ALERT_API_PASSWORD},
    timeout=5,
).json()['access_token']

api_client = Client(token, DISTANT_API_URL)

# get cameras -> in order to get cam name from cam id
cameras = api_client.fetch_cameras().json()

# define base directory
base_dir = os.path.join(BASE_DIRECTORY)
os.makedirs(base_dir, exist_ok=True)


In [ ]:
for seq_id in SEQUENCE_ID_LIST: 
    sequences = api_client.fetch_sequences_detections(sequence_id=seq_id).json()

    cam_name = [item['name'] for item in cameras if item['id'] == sequences[0]['camera_id']][0]
    created_at_rounded = sequences[0]["created_at"].split('.')[0].replace(':', '-').replace('T', '_')
    
    print(f"== Download Alerts data for sequence ID {seq_id} - camera {cam_name} at {created_at_rounded}")
    
    alert_dir = os.path.join(f"{cam_name}_{created_at_rounded}")
    image_dir = os.path.join(base_dir,alert_dir,  "images")
    pred_dir = os.path.join(base_dir, alert_dir, "labels_predictions")
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(pred_dir, exist_ok=True)

    for seq in sequences:
        
        # bbox
        #yolo_format_bbox = ' '.join(map(str,ast.literal_eval(seq["bboxes"])[0]))
        bboxes = seq["bboxes"]

        bbox_file_name = seq["bucket_key"][:-4] + ".txt"
        
        with open(os.path.join(base_dir, alert_dir, "labels_predictions",bbox_file_name), 'w') as f:
            f.write(bboxes)

        url = seq['url']
        nom_fichier = seq["bucket_key"]
        # image
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            full_img_path = os.path.join(image_dir, nom_fichier)
            with open(full_img_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
        else:
            print(f"Error during download.")